## First step - generate training data from examples

In [105]:
import requests
import pandas as pd
from datetime import datetime, timedelta,timezone
from datasets import load_dataset, Dataset
from dotenv import load_dotenv
import os
from openai import OpenAI
import json
import tiktoken
from IPython.display import display, Markdown



In [96]:
# Load environment variables in a file called .env
load_dotenv()

# Replace with your CoinAPI key
API_KEY = os.getenv('YOUR_COINAPI_KEY')

# Define the base URL for CoinAPI
BASE_URL = 'https://rest.coinapi.io/v1/ohlcv/'
OLLAMA_URL = "http://localhost:11434/v1"

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
# URL to fetch the OHLCV data


In [98]:
openai = OpenAI()

ollama = OpenAI(
    base_url=OLLAMA_URL,
    api_key='OLAMMA'
)


In [104]:
# Define the symbol and timeframe
base_data = {
    'name': 'Cardano',
    'symbol': f'BINANCE_SPOT_ADA_USDT',
    'timeframe': '1DAY',
    'time_range': 365 * 2
}


# Calculate the start date for one year ago
end_date = datetime.now(tz=timezone.utc)

start_date = end_date - timedelta(days=base_data['time_range'])

# Format the dates in the required format (ISO 8601)
start_date_str = start_date.strftime('%Y-%m-%dT%H:%M:%S')
end_date_str = end_date.strftime('%Y-%m-%dT%H:%M:%S')

# Headers for authentication
headers = {
    'X-CoinAPI-Key': API_KEY
}

# URL to fetch the OHLCV base_data
url = f'{BASE_URL}{base_data['symbol']}/history'

# Request parameters
params = {
    'period_id': base_data['timeframe'],
    'time_start': start_date_str,
    'time_end': end_date_str,
    'limit': 1000  # Maximum number of records per request
}

In [91]:
# Fetch the data
response = requests.get(url, headers=headers, params=params)     
len(response.json())

729

In [76]:
# Check for successful response
if response.status_code == 200:
    data = response.json()

    if data:
        # Convert to DataFrame for better readability
        df = pd.DataFrame(data)

        df = df[["time_period_start", "price_open", "price_high", "price_low", "price_close"]]
        df.columns = ["Timestamp", "Open", "High", "Low", "Close"]

        # Convert timestamp to readable format
        df["Timestamp"] = pd.to_datetime(df["Timestamp"]).dt.strftime("%Y-%m-%d")

        # Display the first few rows of the data
        print(df.tail())
        
        # Convert last 365 days of data into JSON format
        price_history = df.to_dict(orient="records")
       
    else:
        print('No data found for the given period.')
else:
    print(f'Error fetching data: {response.status_code}, {response.text}')

      Timestamp    Open    High     Low   Close
724  2025-02-06  0.7325  0.7660  0.6978  0.7052
725  2025-02-07  0.7052  0.7532  0.6902  0.7072
726  2025-02-08  0.7072  0.7180  0.6815  0.7005
727  2025-02-09  0.7006  0.7160  0.6503  0.6814
728  2025-02-10  0.6815  0.7177  0.6632  0.7037


In [47]:
def count_tokens(text, model="gpt-4o"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


In [ ]:
  # Construct prompt 

prompt = f"""
  Given the last 365 days of ${base_data['name']} OHLC data:

  {json.dumps(price_history, indent=2)}

  Analyze this data and provide a trading signal (Buy, Sell, or Hold) for today based on the trend and the price action.
  Note that today is {end_date.strftime('%Y-%m-%d')}
  Also, provide short term ,mid term and long term signals.
  """
num_tokens = count_tokens(prompt)
print(f"Estimated Tokens: {num_tokens}")

print(prompt)

Estimated Tokens: 40515

Given the last 365 days of $Cardano OHLC data:

[
  {
    "Timestamp": "2023-02-12",
    "Open": 0.3685,
    "High": 0.3738,
    "Low": 0.3596,
    "Close": 0.3641
  },
  {
    "Timestamp": "2023-02-13",
    "Open": 0.3641,
    "High": 0.3657,
    "Low": 0.345,
    "Close": 0.3593
  },
  {
    "Timestamp": "2023-02-14",
    "Open": 0.3593,
    "High": 0.3931,
    "Low": 0.3545,
    "Close": 0.3867
  },
  {
    "Timestamp": "2023-02-15",
    "Open": 0.3868,
    "High": 0.42,
    "Low": 0.3819,
    "Close": 0.419
  },
  {
    "Timestamp": "2023-02-16",
    "Open": 0.419,
    "High": 0.4213,
    "Low": 0.3862,
    "Close": 0.3874
  },
  {
    "Timestamp": "2023-02-17",
    "Open": 0.3874,
    "High": 0.4098,
    "Low": 0.3854,
    "Close": 0.4033
  },
  {
    "Timestamp": "2023-02-18",
    "Open": 0.4032,
    "High": 0.4184,
    "Low": 0.3976,
    "Close": 0.406
  },
  {
    "Timestamp": "2023-02-19",
    "Open": 0.406,
    "High": 0.4122,
    "Low": 0.3968,
    "

In [ ]:
def get_response(prompt):
    new_response = ollama.chat.completions.create(model="llama3.2",
    messages=[
        {"role": "system", "content": f"You are a trading analyst providing Buy/Sell/Hold signals based on ${base_data['name']} price history.Note that today is {end_date.strftime('%Y-%m-%d')}"},
        {"role": "user", "content": prompt}
    ],
    stream=True,
    max_tokens=5500
    )
    markdown_content = ""
    
    # Stream response and accumulate markdown content
    for chunk in new_response:
        content = chunk.choices[0].delta.content or ''
        markdown_content += content
        
        # Clear output and display updated markdown
        display(Markdown(markdown_content), clear=True)
        
        yield content

# Call the function and consume the generator to start streaming
for _ in get_response(prompt):
    pass

To analyze this data, I'll use a combination of moving averages, relative strength index (RSI), and other technical indicators. Please note that this is a simplified analysis and should not be considered as professional trading advice.

**Current Data**

For 2025-02-10, the opening price is not available. However, we can calculate the current prices based on the historical data provided.

Let's assume the last known close price for 2025-02-09 was $0.6815. For simplicity, let's use this as the opening price for today (2025-02-10).

**Short-Term Signal**

For a short-term signal, I'll use a simple moving average crossover system.

* Short-Term Moving Average (20 days): $0.6922
* Short-Term Moving Average (10 days): $0.6747

Since the 20-day MA ($0.6922) is above the 10-day MA ($0.6747), we can conclude that **Buy** in this timeframe.

**Mid-Term Signal**

For a mid-term signal, I'll use RSI.

* Current Price: $0.6815
* Overbought Region: 70-80
* Oversold Region: 20-50

The current price ($0.6815) is at the lower end of the oversold region (20-50), indicating a potential buying opportunity.

Since RSI values are not provided for the entire dataset, we'll use an RSI value of 30 (midpoint of the low and high values). At $0.6815, RSI is approximately 34.

* Mid-Term Moving Average: Not available
* Mid-Term Momentum: Rising

Considering the oversold region and rising momentum, **Hold** is a reasonable mid-term strategy for today.

**Long-Term Signal**

For a long-term signal, I'll use the overall trend direction based on historical data.

The dataset shows an upward trend (average True Range, AtR, value has been increasing). From 2025-02-03 to 2025-02-09, there were 6 consecutive increases in this dataset. That's a strong positive trend.

Since there are no obvious signs of weakness in the long-term data or divergence with other trends (like 50-day MA), I recommend **Hold** for an extended holding period, keeping an eye on RSI values and adjusting positions as needed to stay ahead of potential price drops.

**Summary**

* Short-Term: **Buy**
* Mid-Term: **Hold**
* Long-Term: **Hold**

In [88]:
def get_response(prompt):
    new_response = openai.chat.completions.create(model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": f"You are a trading analyst providing Buy/Sell/Hold signals based on ${base_data['name']} price history. Format your response in markdown.Note that today is {end_date.strftime('%Y-%m-%d')}"},
        {"role": "user", "content": prompt}
    ],
    stream=True,
    max_tokens=5500
    )
    
    # Initialize markdown cell output
    markdown_content = ""
    
    # Stream response and accumulate markdown content
    for chunk in new_response:
        content = chunk.choices[0].delta.content or ''
        markdown_content += content
        
        # Clear output and display updated markdown
        display(Markdown(markdown_content), clear=True)
        
        yield content

# Call the function and consume the generator to start streaming
for _ in get_response(prompt):
    pass

# $Cardano Trading Analysis for 2025-02-10

### **Current Price Analysis**
- **Open:** 0.6815
- **High:** 0.7177
- **Low:** 0.6632
- **Close:** 0.7037

The price of $Cardano closed 3.59% higher than the previous day's close. This suggests a potential bullish reversal following a downward trend observed over the last few days. However, the volatility in the high-low range reflects uncertainty in the market.

### **Trend Overview**
- **Short-term:** 
  - The recent price action indicates a possible recovery as we see an upward close. The price is currently attempting to break resistance, but the last few days exhibited mixed movements (e.g., a decrease before the recent increase). 
- **Mid-term:**
  - Over the past month, $Cardano has experienced significant volatility. While it reached its peak at around 1.079 earlier in January, the subsequent decline indicates selling pressure in the mid-term. A consolidation phase appears as buyers are trying to push the price back up.
- **Long-term:**
  - Over the past year, $Cardano has shown high volatility and a fluctuating price range, but it has generally been trending downwards since its recent highs. 

### **Trading Signals**
- **Short-term Signal:** **Buy**
  - The recent upward price movement along with a closing above 0.7000 indicates potential upward momentum. Short-term traders may consider buying into this recovery signal.

- **Mid-term Signal:** **Hold**
  - Within the last month, while recovery is in place, it is prudent to wait for confirmation of sustained upward movement before committing larger positions. A hold is advised to monitor the situation.

- **Long-term Signal:** **Sell**
  - Given that the longer-term trends show a downward trajectory since peaking at higher prices, long-term holders might consider selling or reducing positions, especially if the price fails to stay above recent resistance levels.

### **Conclusion**
Today’s price action indicates a bullish sentiment in the short term but still reflects uncertainty in the mid and long-term periods. It would be wise for traders to remain cautious and adjust positions as the market dynamics evolve further. Always consider your risk management strategies when deciding to enter or exit positions.